In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with the Vertex AI Gemini API & Python SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/search_data_blending_with_gemini_summarization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fsearch_data_blending_with_gemini_summarization.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/search_data_blending_with_gemini_summarization.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/search/search_data_blending_with_gemini_summarization.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>


## Overview

### Search
Vertex AI Search brings together the power of deep information retrieval, state-of-the-art natural language processing, and the latest in large language processing to understand user intent and return the most relevant results for the user.

With Vertex AI Search, you can create apps for searching and for making recommendations. Vertex AI Search also has special capabilities for some industries, such as media, healthcare, and retail.



### Gemini

Gemini is a family of generative AI models developed by Google DeepMind that is designed for multimodal use cases. The Gemini API gives you access to the Gemini Pro and Gemini Pro Vision models.

### Vertex AI Gemini API

The Vertex AI Gemini API provides a unified interface for interacting with Gemini models. There are two Gemini 1.0 Pro models available in the Gemini API:

- **Gemini 1.0 Pro model** (`gemini-1.0-pro`): Designed to handle natural language tasks, multi-turn text and code chat, and code generation.
.

You can interact with the Gemini API using the following methods:

- Use [Vertex AI Studio](https://cloud.google.com/generative-ai-studio) for quick testing and command generation
- Use cURL commands
- Use the Vertex AI SDK

For more information, see the [Generative AI on Vertex AI](https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview) documentation.

This tutorial explains how to call a search app with mixed datastore, get search snippets and summarize the response using Gemini Pro.


### Create a Search App with Mixed Datastores

1. Follow the steps listed here to create a Search App
   - https://cloud.google.com/generative-ai-app-builder/docs/create-engine-es
2. Create the relevant data stores (GCS, BQ, Website)
   - https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es
3. Link the data stores to the Search App
   - https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#multi-data-stores

The example query and results are based on the data used in this tutorial:
   - https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search

### Install the Relevant packages

In [ ]:
%pip install --upgrade --user -q google-cloud-aiplatform google-cloud-discoveryengine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.1 MB/s eta 0:00:00


### Restart current runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which will restart the current kernel.

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Authenticate your notebook environment (Colab only)

If you are running this notebook on Google Colab, run the following cell to authenticate your environment. This step is not required if you are using [Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench).


In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Define Google Cloud project information

In [ ]:
PROJECT_ID = ""  # @param {type:"string"}
SEARCH_APP_LOCATION = "global"  # @param {type:"string"}
SEARCH_ENGINE_ID = ""  # @param {type:"string"}
LOCATION_GEMINI_MODEL = "us-central1"  # @param {type:"string"}

### Initialize the Vertex AI SDK

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION_GEMINI_MODEL)

### Import the Relevant packages

In [ ]:
from typing import List
import re

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1alpha as discoveryengine

from vertexai.generative_models import (
    GenerativeModel,
    GenerationConfig,
    HarmCategory,
    HarmBlockThreshold,
)

### Send a Request to Vertex AI Search App with Data Blending (Mixed Datastore)

- https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es#multi-data-stores

In [ ]:
search_query = "What was Google cloud's revenue in Q4 2020?"

In [ ]:
# Create a client using a regional endpoint
client = discoveryengine.SearchServiceClient(
    client_options=(
        ClientOptions(
            api_endpoint=f"{SEARCH_APP_LOCATION}-discoveryengine.googleapis.com"
        )
        if SEARCH_APP_LOCATION != "global"
        else None
    )
)

# The full resource name of the search app serving config
serving_config = f"projects/{PROJECT_ID}/locations/{SEARCH_APP_LOCATION}/collections/default_collection/engines/{SEARCH_ENGINE_ID}/servingConfigs/default_config"



response = client.search(
    discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=5,
    )
)

### Extract & clean up snippets from search results


In [ ]:
response

SearchPager<results {
  id: "217e8bedecfe08e3c43f5b289af15243"
  document {
    name: "projects/9452062936/locations/global/collections/default_collection/dataStores/google-financials_1710902168241/branches/0/documents/217e8bedecfe08e3c43f5b289af15243"
    id: "217e8bedecfe08e3c43f5b289af15243"
    derived_struct_data {
      fields {
        key: "extractive_answers"
        value {
          list_value {
            values {
              struct_value {
                fields {
                  key: "content"
                  value {
                    string_value: "Google succeeds when we help our customers and partners succeed, and we see significant opportunities to forge meaningful partnerships as businesses increasingly look to a digital future.\342\200\235 Ruth Porat, CFO of Google and Alphabet, said: \342\200\234Our strong fourth quarter performance, with revenues of $56.9 billion, was driven by Search and YouTube, as consumer and business activity recovered from earlier i

In [ ]:
retrieved_data: List[str] = []

for result in response.results:
    data = result.document.derived_struct_data
    print(data)
    if not data:
        continue



    snippets: List[str] = [
        re.sub("<[^>]*>", "", snippet_item.get("content", ""))
        for snippet_item in data.get("extractive_answers", [])
        if snippet_item.get("content")
    ]



    # snippets: List[str] = [
    #     re.sub("<[^>]*>", "", snippet_item.get("snippet", ""))
    #     for snippet_item in data.get("snippets", [])
    #     if snippet_item.get("snippet")
    # ]

    if snippets:
        title = data.get("title", "Unknown Title")
        retrieved_data.append(
            f"--- Snippets from Document {title} ---\n{''.join(snippets)}\n"
        )

### Feed the Search result snippets to Gemini Pro model and formuate a summary/response based on your original prompt

#### Model parameters

Every prompt you send to the model includes parameter values that control how the model generates a response.

The model can generate different results for different parameter values.

You can experiment with different model parameters to see how the results change.


In [ ]:
generation_config = GenerationConfig(
    temperature=0,
    top_p=1.0,
    max_output_tokens=2048,
)

In [ ]:
# Prompt for Gemini Pro model
PROMPT_GEMINI = f"""Provide an answer to the question based on the information in the Document snippets provided with citations.
Question: {search_query}
{''.join(retrieved_data)}
"""

model = GenerativeModel("gemini-1.0-pro")  # specify the gemini model version


def generate(PROMPT_GEMINI: str) -> str:
    """
    Given the prompt
    output the summarized response to user's orginal query
    """
    responses = model.generate_content(
        PROMPT_GEMINI,
        generation_config=generation_config,
        safety_settings={
            HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
            HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
        },
        stream=True,
    )

    for response in responses:
        print(response.text, end="")


print(f"PROMPT:\n{PROMPT_GEMINI}")

print("Gemini Response:\n")
generate(PROMPT_GEMINI)

PROMPT:
Provide an answer to the question based on the information in the Document snippets provided with citations.
Question: What was Google cloud's revenue in Q4 2020?
--- Snippets from Document GOOG Exhibit 99.1 Q4'20 ---
Google succeeds when we help our customers and partners succeed, and we see significant opportunities to forge meaningful partnerships as businesses increasingly look to a digital future.” Ruth Porat, CFO of Google and Alphabet, said: “Our strong fourth quarter performance, with revenues of $56.9 billion, was driven by Search and YouTube, as consumer and business activity recovered from earlier in the year.
--- Snippets from Document GOOG 10-K Q4 2020 ---
Year Ended December 31, 2019 2020 Google Search &amp; other $ 98115 $ 104062 YouTube ads 15149 19772 Google Network Members&#39; properties 21547 23090 Google advertising 134811 146924 Google other 17014 21711 Google Services total 151825 168635 Google Cloud 8918 13059 Other Bets 659 657 Hedging gains (losses) 45